## Plan for region-spanning river basins
We suspect that some GRDC major river basins will cross RGI regions.  This promises to complicate data processing.  But how many such basins are there, and which ones are they?

This notebook uses the RGI IDs listed in Erik Holmgren's lookup table to define which GRDC basins contain glaciers from more than one RGI region.

19 Jun 2023 | EHU

In [22]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import date
import collections
import datetime
import json
import os

In [23]:
def select_glaciers_json(basin='all'):
    '''
    Select glaciers within a basin by MRBID from a json-file,
    which is stored in the data directory.

    Args:
    -----
    basin: str
        String of MRBID or 'all'.

    Returns:
    --------
    If basin is 'all' a list of all relevant glaciers is returned, for
    initiating glacier simulations. If basin is a MRBID the list of glaciers
    within that basin is returned.
    
    Copy of a function written by Erik Holmgren (2022) in holmgren_gha.utils
    '''

    # fpath = './data/rgi_ids_per_basin.json'
    fpath = '/Users/lizz/Documents/Research/Runoff-intercomparison/msc_thesis-multi_gcm/code/data/rgi_ids_per_basin.json' ## correct for local run
    with open(fpath) as f:
        basin_dict = json.load(f)

    if basin.lower() != 'all':
        glacier_list = basin_dict[basin]
    else:
        glacier_list = list(itertools.chain.from_iterable(basin_dict.values()))

    return glacier_list

In [24]:
## which basins do we study?  List from Lily's output structure
all_basins = {2103: 'INDIGIRKA',
              2108: 'OB',
              2302: 'BRAHMAPUTRA',
              2306: 'GANGES',
              2309: 'INDUS',
              2310: 'IRRAWADDY',
              2319: 'SALWEEN',
              2413: 'KAMCHATKA',
              2421: 'MEKONG',
              2433: 'YANGTZE',
              2434: 'YELLOW RIVER',
              2902: 'ARAL SEA', ## endorheic from here to issyk-kul
              2905: 'CHUY',
              2909: 'HAR US NUUR',
              2910: 'LAKE BALKHASH',
              2913: 'TALAS',
              2914: 'TARIM HE',
              2918: 'UVS NUUR',
              2919: 'ISSYK-KUL', ## endorheic
              3203: 'AMAZON',
              3209: 'CHICO',
              3212: 'COLORADO',
              3227: 'MAGDALENA',
              3232: 'NEGRO',
              3244: 'SANTA CRUZ',
              3401: 'AISEN',
              3403: 'AZOPARDO', ## note only 3541 km3
              3404: 'BAKER',
              3405: 'BIOBIO',
              3408: 'CISNES',
              3409: 'COPIAPO',
              3412: 'HUASCO',
              3416: 'MAJES',
              3418: 'OCONA',
              3419: 'PALENA',
              3420: 'PASCUA',
              3422: 'PUELO',
              3423: 'RAPEL',
              3425: 'SANTA',
              3426: 'SERRANO',
              3428: 'VALDIVIA',
              3429: 'YELCHO',
              3912: 'TITICACA', ## endorheic!
              4110: 'COLVILLE',
              4123: 'MACKENZIE',
              4125: 'NELSON',
              4401: 'ALSEK',
              4406: 'COLUMBIA',
              4408: 'COPPER',
              4410: 'FRASER',
              4414: 'KUSKOKWIM',
              4416: 'NASS',
              4418: 'NUSHAGAK',
              4426: 'SKAGIT',
              4427: 'SKEENA',
              4428: 'STIKINE',
              4430: 'SUSITNA',
              4431: 'TAKU',
              4435: 'YUKON',
              5406: 'CLUTHA', ## NEW ZEALAND?
              6101: 'JOKULSA A FJOLLUM',
              6104: 'LAGARFLIOT', ## note only 3310 km3
              6110: 'SVARTA', ## note only 3571 km3
              6202: 'DANUBE',
              6209: 'DRAMSELVA',
              6213: 'GLOMAA',
              6219: 'KALIXALVEN',
              6223: 'KUBAN',
              6227: 'LULEALVEN',
              6237: 'OLFUSA',
              6241: 'PO',
              6242: 'RHINE',
              6243: 'RHONE',
              6254: 'THJORSA',
              6255: 'TORNEALVEN'
             }

In [25]:
summary_df = pd.DataFrame.from_dict(all_basins, orient='index', columns=['BasinName'])
summary_df['RGIregions'] = np.empty(len(all_basins))
summary_df

,BasinName,RGIregions
2103,INDIGIRKA,2.858498e-314
2108,OB,2.858495e-314
2302,BRAHMAPUTRA,2.858497e-314
2306,GANGES,2.858498e-314
2309,INDUS,2.858498e-314
...,...,...
6241,PO,2.855480e-314
6242,RHINE,2.855480e-314
6243,RHONE,2.855480e-314
6254,THJORSA,2.855480e-314


In [26]:
for mrbid in summary_df.index:
    glaciers_list = select_glaciers_json(str(mrbid)) ## list all the glaciers in this basin
    regions = set([np.floor(float(s.replace('RGI60-', ''))) for s in glaciers_list]) ## pull out the unique region tags
    summary_df.at[mrbid, 'RGIregions'] = regions
    if len(regions)>1:
        print(all_basins[mrbid], regions)

BRAHMAPUTRA {13.0, 15.0}
INDUS {13.0, 14.0}
SALWEEN {13.0, 15.0}
YANGTZE {13.0, 15.0}
ARAL SEA {13.0, 14.0}
TARIM HE {13.0, 14.0}
YUKON {1.0, 2.0}


There are only a handful of basins that span RGI regions...but unfortunately, they include some that we find very interesting (such as Indus, Aral Sea, and Tarim).  Most of these are in Central Asia.  EHU make a plan to process these and export aggregated data.